In [1]:
import tensorflow
import numpy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import preprocessing
from tensorflow.keras.layers import  Embedding, Dense, Flatten, LSTM, Dropout, BatchNormalization, Bidirectional, Conv1D, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import io
import json
import os

In [2]:
samples = numpy.array([])
ids = numpy.array([])
with open('train.txt') as text:
    inc = 0
    lines = text.readlines()
    lines = list(dict.fromkeys(lines))
    for line in lines:
        inc = inc + 1
        if inc == 10000:
            break
        #print(line.split('\\'))
        splittedLines = line.split('\\')
        if len(splittedLines) != 1 and splittedLines[0] != 'TradeBot':
            samples = numpy.append(samples, splittedLines[1])
            ids = numpy.append(ids, ''.join(e for e in splittedLines[0] if e.isalnum()))

In [3]:
tokenizer2 = Tokenizer(num_words=20000)
tokenizer2.fit_on_texts(samples)
tokenizer2ForNames = Tokenizer(num_words=2000)
tokenizer2ForNames.fit_on_texts(ids)

# saving
tokenizer_json = tokenizer2.to_json()
with io.open('tokenizer2.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

tokenizerForNames_json = tokenizer2ForNames.to_json()
with io.open('tokenizer2ForNames.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizerForNames_json, ensure_ascii=False))


In [4]:
sequences = tokenizer2.texts_to_sequences(samples)
tokenizedNames = tokenizer2ForNames.texts_to_sequences(ids)
len(sequences) - len(tokenizedNames)
giallarToken = tokenizer2ForNames.texts_to_sequences([['Giallar']])[0][0]
sequences = preprocessing.sequence.pad_sequences(sequences, maxlen=10)


In [5]:
maxnumofmessages = 5
x_train = None
y_train = numpy.array([])
for i in range(len(tokenizedNames) - maxnumofmessages - 1):
    prehistory = numpy.array([])
    for j in range(maxnumofmessages):
        prehistory = numpy.append(prehistory, sequences[i + j - 1])
        prehistory = numpy.array([prehistory.flatten()])

    if x_train is None:
        x_train = numpy.array(prehistory)
    else:
        x_train = numpy.append(x_train, prehistory, axis=0)

    if tokenizedNames[i + maxnumofmessages][0] == giallarToken:
        y_train = numpy.append(y_train, 1)
    else:
        y_train = numpy.append(y_train, 0)

In [41]:
prehistory

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 7.084e+03, 2.720e+02, 3.000e+01, 1.575e+03,
        4.870e+02, 6.930e+02, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 2.940e+02,
        1.610e+02, 1.370e+02, 4.810e+02, 7.085e+03, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        6.000e+01, 2.500e+01, 7.086e+03, 3.000e+01, 5.000e+00, 3.590e+02,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 6.000e+00, 1.200e+01, 3.630e+02,
        7.087e+03, 7.088e+03, 0.000e+0

In [42]:
x_train

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        4.290e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 1.318e+03,
        2.278e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 5.000e+02, 2.281e+03,
        2.282e+03],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 3.000e+01, 5.000e+00,
        3.590e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 3.630e+02, 7.087e+03,
        7.088e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 3.870e+02, 1.200e+01,
        1.100e+03]])

In [13]:
y_train

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [9]:
model = Sequential()
model.add(Embedding(20000, 64,input_length=50))
model.add(Conv1D(32, 8, activation='relu'))
model.add(MaxPooling1D(5))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(10, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy')
metrics=['acc']
model.summary()

checkpoint_path = "training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 64)            1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 43, 32)            16416     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 32)             0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 32)             128       
_________________________________________________________________
dropout_6 (Dropout)          (None, 8, 32)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 8, 128)            49664     
_________________________________________________________________
batch_normalization_7 (Batch (None, 8, 128)           

In [7]:
x_train[0]

array([    0.,  1354., 12179.,   379., 12180.,  3080.,   790., 12181.,
         669., 12182.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,  4188.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,  4189.,  4190.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,   721.])

In [12]:
history = model.fit(
                    x_train,
                    y_train,
                    epochs=6,
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=[cp_callback])

Epoch 1/6
122/122 [==============================] - ETA: 0s - loss: 0.0933

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xce in position 160: invalid continuation byte

In [29]:
model.predict(numpy.array([x_train[0]]))

array([[0.9999996]], dtype=float32)

In [30]:
model.predict(x_train)



array([[9.99999642e-01],
       [8.08872400e-08],
       [2.25836203e-08],
       [7.58453123e-09],
       [2.39672393e-09],
       [8.61634042e-10],
       [3.47704443e-09],
       [3.10153192e-09],
       [5.59416824e-09],
       [1.42329366e-08],
       [1.17781518e-09],
       [1.60465252e-09],
       [1.78168524e-09],
       [2.33822600e-10],
       [5.07565268e-10],
       [1.21837329e-09],
       [1.60601821e-09],
       [1.05697151e-09],
       [2.64265942e-09],
       [9.62031610e-09],
       [2.53395704e-09],
       [8.35865777e-09],
       [2.65748956e-09],
       [4.75085915e-10],
       [1.12496001e-09],
       [9.18333742e-10],
       [1.29291833e-09],
       [3.05249226e-09],
       [3.80705032e-08],
       [1.00000000e+00],
       [4.24248681e-09],
       [2.51658849e-09],
       [5.49523260e-10],
       [1.49272594e-08],
       [1.45746863e-08],
       [6.16860385e-09],
       [2.45566745e-09],
       [1.00000000e+00],
       [2.70540568e-07],
       [9.99999762e-01],
